# Library import

In [1]:
# 필요 library들을 import합니다.
import os
from typing import Tuple, Any, Callable, List, Optional, Union

import cv2
import timm
import torch
import numpy as np
import pandas as pd
import albumentations as A
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import models, datasets, transforms
from tqdm.auto import tqdm
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from albumentations.pytorch import ToTensorV2

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Dataset Class

In [ ]:
def add_padding(image, target_size=(224, 224)):
    original_height, original_width = image.shape[:2]
    
    # 목표 크기
    target_height, target_width = target_size
    
    # 비율 계산
    ratio = min(target_width / original_width, target_height / original_height)
    new_width = int(original_width * ratio)
    new_height = int(original_height * ratio)
    
    # 이미지 리사이즈
    resized_image = cv2.resize(image, (new_width, new_height), interpolation=cv2.INTER_AREA)

    # 패딩 계산
    delta_w = target_width - new_width  # 가로 패딩 총합
    delta_h = target_height - new_height  # 세로 패딩 총합
    padding_left = delta_w // 2
    padding_top = delta_h // 2
    padding_right = delta_w - padding_left
    padding_bottom = delta_h - padding_top
    
    # 패딩 추가 (흰색으로 채우기)
    transform = A.Compose([
        A.PadIfNeeded(min_height=target_height, min_width=target_width, border_mode=255)  # border_mode=255는 흰색
    ])
    
    # 패딩 적용
    padded_image = transform(image=resized_image)['image']
    
    return padded_image

In [2]:
class CustomDataset(Dataset):
    def __init__(
        self,
        root_dir: str,
        info_df: pd.DataFrame,
        transform: Callable,
        is_inference: bool = False
    ):
        # 데이터셋의 기본 경로, 이미지 변환 방법, 이미지 경로 및 레이블을 초기화합니다.
        self.root_dir = root_dir  # 이미지 파일들이 저장된 기본 디렉토리
        self.transform = transform  # 이미지에 적용될 변환 처리
        self.is_inference = is_inference # 추론인지 확인
        self.image_paths = info_df['image_path'].tolist()  # 이미지 파일 경로 목록

        if not self.is_inference:
            self.targets = info_df['target'].tolist()  # 각 이미지에 대한 레이블 목록

    def __len__(self) -> int:
        # 데이터셋의 총 이미지 수를 반환합니다.
        return len(self.image_paths)

    def __getitem__(self, index: int) -> Union[Tuple[torch.Tensor, int], torch.Tensor]:
        # 주어진 인덱스에 해당하는 이미지를 로드하고 변환을 적용한 후, 이미지와 레이블을 반환합니다.
        img_path = os.path.join(self.root_dir, self.image_paths[index])  # 이미지 경로 조합
        image = cv2.imread(img_path, cv2.IMREAD_COLOR)  # 이미지를 BGR 컬러 포맷의 numpy array로 읽어옵니다.
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # BGR 포맷을 RGB 포맷으로 변환합니다.
        image = self.transform.resize_with_padding(image)
        image = self.transform(image)  # 설정된 이미지 변환을 적용합니다.
        

        if self.is_inference:
            return image
        else:
            target = self.targets[index]  # 해당 이미지의 레이블
            return image, target  # 변환된 이미지와 레이블을 튜플 형태로 반환합니다.

# Transform Class

In [3]:
class TorchvisionTransform:
    def __init__(self, is_train: bool = True):
        # 공통 변환 설정: 이미지 리사이즈, 텐서 변환, 정규화
        common_transforms = [
            transforms.Resize((224, 224)),  # 이미지를 224x224 크기로 리사이즈
            transforms.ToTensor(),  # 이미지를 PyTorch 텐서로 변환
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # 정규화
        ]

        if is_train:
            # 훈련용 변환: 랜덤 수평 뒤집기, 랜덤 회전, 색상 조정 추가
            self.transform = transforms.Compose(
                [
                    transforms.RandomHorizontalFlip(p=0.5),  # 50% 확률로 이미지를 수평 뒤집기
                    transforms.RandomRotation(15),  # 최대 15도 회전
                    transforms.ColorJitter(brightness=0.2, contrast=0.2),  # 밝기 및 대비 조정
                ] + common_transforms
            )
        else:
            # 검증/테스트용 변환: 공통 변환만 적용
            self.transform = transforms.Compose(common_transforms)

    def __call__(self, image: np.ndarray) -> torch.Tensor:
        image = Image.fromarray(image)  # numpy 배열을 PIL 이미지로 변환

        transformed = self.transform(image)  # 설정된 변환을 적용

        return transformed  # 변환된 이미지 반환

In [4]:
class AlbumentationsTransform:
    def __init__(self, is_train: bool = True):
        # 공통 변환 설정: 이미지 리사이즈, 정규화, 텐서 변환
        common_transforms = [
   #         A.Resize(384, 384),  # 이미지를 224x224 크기로 리사이즈
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # 정규화
            ToTensorV2()  # albumentations에서 제공하는 PyTorch 텐서 변환
        ]

        if is_train:
            # 훈련용 변환: 랜덤 수평 뒤집기, 랜덤 회전, 랜덤 밝기 및 대비 조정 추가
            self.transform = A.Compose(
                [
                    A.HorizontalFlip(p=0.5),  # 50% 확률로 이미지를 수평 뒤집기
                    A.Rotate(limit=15, p=0.5),  # 최대 15도 회전
                    A.Affine(scale=(1, 1.5), shear=(-10, 10), p=0.5),
                    A.ElasticTransform(alpha=10, sigma=50, p=0.5),
                    A.RandomBrightnessContrast(p=0.5),  # 밝기 및 대비 무작위 조정
                    A.MotionBlur(blur_limit=(3, 7), p=0.5),
                    A.CoarseDropout(p=0.5, max_holes=8, min_height=(30), max_height=(50), min_width=30, max_width=50, fill_value=255),
                ] + common_transforms
            )
        else:
            # 검증/테스트용 변환: 공통 변환만 적용
            self.transform = A.Compose(common_transforms)

    def resize_with_padding(self, image, target_size=(384, 384)):
        original_height, original_width = image.shape[:2]
        target_height, target_width = target_size

        # 비율 계산
        ratio = min(target_width / original_width, target_height / original_height)
        new_width = int(original_width * ratio)
        new_height = int(original_height * ratio)

        # 이미지 리사이즈 (비율 유지)
        resize_transform = A.Resize(height=new_height, width=new_width)
        resized_image = resize_transform(image=image)['image']

        # 패딩 추가
        pad_transform = A.PadIfNeeded(min_height=target_height, min_width=target_width, border_mode=255)  # 흰색 패딩
        padded_image = pad_transform(image=resized_image)['image']

        return padded_image

    def __call__(self, image) -> torch.Tensor:
        # 이미지가 NumPy 배열인지 확인
        if not isinstance(image, np.ndarray):
            raise TypeError("Image should be a NumPy array (OpenCV format).")

        # 이미지에 변환 적용 및 결과 반환
        transformed = self.transform(image=image)  # 이미지에 설정된 변환을 적용

        return transformed['image']  # 변환된 이미지의 텐서를 반환

In [5]:
class TransformSelector:
    """
    이미지 변환 라이브러리를 선택하기 위한 클래스.
    """
    def __init__(self, transform_type: str):

        # 지원하는 변환 라이브러리인지 확인
        if transform_type in ["torchvision", "albumentations"]:
            self.transform_type = transform_type

        else:
            raise ValueError("Unknown transformation library specified.")

    def get_transform(self, is_train: bool):

        # 선택된 라이브러리에 따라 적절한 변환 객체를 생성
        if self.transform_type == 'torchvision':
            transform = TorchvisionTransform(is_train=is_train)

        elif self.transform_type == 'albumentations':
            transform = AlbumentationsTransform(is_train=is_train)

        return transform

# Model Class

In [6]:
# 학습에 사용할 장비를 선택.
# torch라이브러리에서 gpu를 인식할 경우, cuda로 설정.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
# 학습 데이터의 경로와 정보를 가진 파일의 경로를 설정.
traindata_dir = "../data/train"
traindata_info_file = "../data/train.csv"
save_result_path = "../train_result"

In [8]:
# 학습 데이터의 class, image path, target에 대한 정보가 들어있는 csv파일을 읽기.
train_info = pd.read_csv(traindata_info_file)

# 총 class의 수를 측정.
num_classes = len(train_info['target'].unique())

# 각 class별로 8:2의 비율이 되도록 학습과 검증 데이터를 분리.
train_df, val_df = train_test_split(
    train_info,
    test_size=0.2,
    stratify=train_info['target']
)

In [9]:
import torch
import numpy as np
import torch.nn.functional as F

def rand_bbox(size, lam):
    """
    Bounding box의 위치를 무작위로 선택.
    :param size: 이미지 크기 (batch_size, channels, height, width)
    :param lam: lambda 값 (patch 비율)
    :return: x1, y1, x2, y2 (bounding box 좌표)
    """
    W = size[2]  # 이미지의 폭
    H = size[3]  # 이미지의 높이
    cut_rat = np.sqrt(1. - lam)  # lambda에 기반한 패치 비율
    cut_w = int(W * cut_rat)
    cut_h = int(H * cut_rat)

    # Bounding box의 중앙 좌표를 무작위로 결정
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    # Bounding box의 좌측 상단과 우측 하단 좌표 계산
    x1 = np.clip(cx - cut_w // 2, 0, W)
    y1 = np.clip(cy - cut_h // 2, 0, H)
    x2 = np.clip(cx + cut_w // 2, 0, W)
    y2 = np.clip(cy + cut_h // 2, 0, H)

    return x1, y1, x2, y2

def mixup_cutmix_collate_fn(batch, alpha=1.0, num_classes=500):
    images, labels = zip(*batch)
    
    # 이미지를 배치 텐서로 변환
    images = torch.stack(images)
    labels = torch.tensor(labels)

    # 원핫 인코딩 (num_classes는 전체 클래스 개수)
    labels = F.one_hot(labels, num_classes=num_classes).float()

    # 0.5 확률로는 아무 것도 적용하지 않음
    prob = np.random.rand()
    
    if prob < 0.5:
        # 아무 증강도 적용하지 않음
        mixed_images, mixed_labels = images, labels
    else:
        # 나머지 0.5 확률 중 절반은 Mixup, 절반은 CutMix
        if prob < 0.75:
            # Mixup 적용
            lam = np.clip(np.random.beta(alpha, alpha), 0.3, 0.7)
            index = torch.randperm(images.size(0))
            mixed_images = images.clone()
            mixed_labels = labels.clone()

            for i in range(images.size(0)):  # 각 이미지에 대해 반복
                mixed_images[i] = lam * images[i] + (1 - lam) * images[index[i]]
                mixed_labels[i] = lam * labels[i] + (1 - lam) * labels[index[i]]
        else:
            # CutMix 적용
            lam = np.clip(np.random.beta(alpha, alpha), 0.3, 0.7)
            index = torch.randperm(images.size(0))
            mixed_images = images.clone()
            mixed_labels = labels.clone()
            
            for i in range(images.size(0)):  # 각 이미지에 대해 반복
                x1, y1, x2, y2 = rand_bbox(images.size(), lam)
                # i번째 이미지에 대해 index[i]번째 이미지의 패치를 적용
                mixed_images[i, :, x1:x2, y1:y2] = images[index[i], :, x1:x2, y1:y2]
                # 라벨도 비율에 따라 섞어줌
                lam_i = 1 - ((x2 - x1) * (y2 - y1) / (images.size(-1) * images.size(-2)))
                mixed_labels[i] = lam_i * labels[i] + (1 - lam_i) * labels[index[i]]

    return mixed_images, mixed_labels



In [10]:
# 학습에 사용할 Transform을 선언.
transform_selector = TransformSelector(
    transform_type = "albumentations"
)
train_transform = transform_selector.get_transform(is_train=True)
val_transform = transform_selector.get_transform(is_train=False)

# 학습에 사용할 Dataset을 선언.
train_dataset = CustomDataset(
    root_dir=traindata_dir,
    info_df=train_df,
    transform=train_transform
)
val_dataset = CustomDataset(
    root_dir=traindata_dir,
    info_df=val_df,
    transform=val_transform
)

# 학습에 사용할 DataLoader를 선언.
train_loader = DataLoader(
    train_dataset,
    batch_size=8,
    shuffle=True,
    num_workers=8,
    collate_fn=mixup_cutmix_collate_fn
)
val_loader = DataLoader(
    val_dataset,
    batch_size=8,
    shuffle=False,
    num_workers=8
)

In [11]:
class SoftTargetCrossEntropy(nn.Module):
    def __init__(self):
        super(SoftTargetCrossEntropy, self).__init__()

    def forward(self, pred: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
        
        log_probs = torch.nn.functional.log_softmax(pred, dim=-1)  # Apply softmax to get log-probabilities
        # Compute loss
        return torch.mean(torch.sum(-target * log_probs, dim=-1))

In [12]:
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from transformers import ConvNextV2ForImageClassification, AutoModelForImageClassification 
import torch.nn.functional as F
from torch.optim.lr_scheduler import CosineAnnealingLR
import timm
import requests

class SwinConvNextClassifier(pl.LightningModule):
    def __init__(self, num_classes=500, lr=1e-4, weight_decay=0.01):
        super().__init__()
       
        self.swin = AutoModelForImageClassification.from_pretrained("team-lucid/swinv2-base-path4-window24-384-doc")
        self.convnext = ConvNextV2ForImageClassification.from_pretrained("facebook/convnextv2-base-22k-384")

        self.swin_output_dim = self.swin.classifier.in_features
        self.swin.classifier = nn.Identity()
     
        
        self.convnext_output_dim = self.convnext.classifier.in_features
        self.convnext.classifier = nn.Identity()

        
        combined_dim = self.swin_output_dim + self.convnext_output_dim

        self.classifier = nn.Linear(combined_dim, num_classes)
 
        self.lr = lr

        self.weight_decay = weight_decay
        self.loss_fn = SoftTargetCrossEntropy()
        self.loss_fn_crossentropy = nn.CrossEntropyLoss()

    def forward(self, pixel_values):
        swin_features = self.swin(pixel_values).logits
        convnet_features = self.convnext(pixel_values).logits

        # Swin과 ConvNexT 특징을 결합 (Concat)
        combined_features = torch.cat((swin_features, convnet_features), dim=1)
        
        # 결합된 특징을 classifier에 통과시켜 최종 출력
        logits = self.classifier(combined_features)
        return logits

    def training_step(self, batch, batch_idx):
        pixel_values, labels = batch
        logits = self.forward(pixel_values)
        loss = self.loss_fn(logits, labels)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        pixel_values, labels = batch
        logits = self.forward(pixel_values)
        loss = self.loss_fn_crossentropy(logits, labels)
        self.log('val_loss', loss)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(
            [
                {'params': self.classifier.parameters(), 'lr': 3e-5, 'weight_decay': 1e-2},  # Classifier에 대한 설정
                {'params': self.convnext.parameters(), 'lr': 1e-5, 'weight_decay': 1e-4},
                {'params': self.swin.parameters(), 'lr': 1e-5, 'weight_decay': 1e-2},
            ]
        )
        scheduler = CosineAnnealingLR(optimizer, T_max=10, eta_min=1e-6)
        return [optimizer], [scheduler]

    def train_dataloader(self):
        # Define train_loader
        return train_loader

    def val_dataloader(self):
        # Define val_loader
        return val_loader

In [13]:
model = SwinConvNextClassifier(num_classes=num_classes, lr=3e-5, weight_decay=1e-2)

Some weights of Swinv2ForImageClassification were not initialized from the model checkpoint at team-lucid/swinv2-base-path4-window24-384-doc and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [14]:
model

SwinConvNextClassifier(
  (swin): Swinv2ForImageClassification(
    (swinv2): Swinv2Model(
      (embeddings): Swinv2Embeddings(
        (patch_embeddings): Swinv2PatchEmbeddings(
          (projection): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
        )
        (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.0, inplace=False)
      )
      (encoder): Swinv2Encoder(
        (layers): ModuleList(
          (0): Swinv2Stage(
            (blocks): ModuleList(
              (0-1): 2 x Swinv2Layer(
                (attention): Swinv2Attention(
                  (self): Swinv2SelfAttention(
                    (continuous_position_bias_mlp): Sequential(
                      (0): Linear(in_features=2, out_features=512, bias=True)
                      (1): ReLU(inplace=True)
                      (2): Linear(in_features=512, out_features=4, bias=False)
                    )
                    (query): Linear(in_features=128, out_features

In [26]:
# 모델 학습
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/swin2convnext2_384_lr=3e-5,padding",  # 체크포인트 저장 경로
    filename="{epoch:02d}-{val_loss:.2f}",  # 저장될 파일명 포맷
    save_top_k=3,  # 상위 몇 개의 모델만 저장할지
    monitor="val_loss",  # 검증 손실을 모니터링하여 체크포인트 저장
    mode="min",  # 손실이 가장 적을 때 저장 (최소화)
    save_weights_only=True  # 전체 모델을 저장 (가중치만 저장하려면 True)
)

trainer = Trainer(max_steps=20000, gradient_clip_val=2, callbacks=checkpoint_callback, accelerator='gpu')
trainer.fit(model)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: Checkpoint directory /data/ephemeral/sketch/checkpoints/swin2convnext2_384 exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                 | Type                             | Params | Mode
---------------------------------------------------------------------------------
0 | swin                 | Swinv2ForImageClassification     | 86.9 M | eval
1 | convnext             | ConvNextV2ForImageClassification | 87.7 M | eval
2 | classifier           | Linear                           | 1.0 M  | eval
3 | loss_fn              | SoftTargetCrossEntropy           | 0      | eval
4 | loss_fn_crossentropy | CrossEntropyLoss                 | 0      | eval
---------------------------------------------------------------------------------
175 M     Trainable params
0

Epoch 3:   1%|▏         | 22/1502 [00:21<24:18,  1.01it/s, v_num=1]        


Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined

# Inference

In [13]:
#model = SwinConvNextClassifier.load_from_checkpoint(checkpoint_path='./checkpoints/swin2convnext2_384/epoch=13-val_loss=0.39.ckpt')

Some weights of Swinv2ForImageClassification were not initialized from the model checkpoint at team-lucid/swinv2-base-path4-window24-384-doc and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [17]:
%ls

bagging.ipynb                              result/
bagging_v2.ipynb                           snapshot.ipynb
beit.ipynb                                 swin.ipynb
beitswin.ipynb                             swin2convnext2_384.ipynb
checkpoints/                               swin_ver2.ipynb
clip-vit-base-patch32.ipynb                swin_ver2_mixup.ipynb
coatnet.ipynb                              swinconvnext.ipynb
convnext.ipynb                             swinconvnext_large.ipynb
convnext2eva02_embedding.ipynb             swinconvnexteva02.ipynb
convnext2eva02_embedding_mixup.ipynb       test.ipynb
convnext2eva02_large_mixupandcutmix.ipynb  tete/
eva02.ipynb                                vit_base_clip_224.ipynb
lightning_logs/


In [14]:
# 모델 추론을 위한 함수
def inference(
    model: nn.Module,
    device: torch.device,
    test_loader: DataLoader
):
    # 모델을 평가 모드로 설정
    model.to(device)
    model.eval()

    predictions = []
    with torch.no_grad():  # Gradient 계산을 비활성화
        for images in tqdm(test_loader):
            # 데이터를 같은 장치로 이동
            images = images.to(device)

            # 모델을 통해 예측 수행
            logits = model(images)
            logits = F.softmax(logits, dim=1)
            preds = logits.argmax(dim=1)

            # 예측 결과 저장
            predictions.extend(preds.cpu().detach().numpy())  # 결과를 CPU로 옮기고 리스트에 추가

    return predictions

In [15]:
# 추론 데이터의 경로와 정보를 가진 파일의 경로를 설정.
testdata_dir = "../data/test"
testdata_info_file = "../data/test.csv"
save_result_path = "../train_result"

In [16]:
# 추론 데이터의 class, image path, target에 대한 정보가 들어있는 csv파일을 읽기.
test_info = pd.read_csv(testdata_info_file)

# 총 class 수.
num_classes = 500

In [17]:
# 추론에 사용할 Transform을 선언.
transform_selector = TransformSelector(
    transform_type = "albumentations"
)
test_transform = transform_selector.get_transform(is_train=False)

# 추론에 사용할 Dataset을 선언.
test_dataset = CustomDataset(
    root_dir=testdata_dir,
    info_df=test_info,
    transform=test_transform,
    is_inference=True
)

# 추론에 사용할 DataLoader를 선언.
test_loader = DataLoader(
    test_dataset,
    batch_size=32,
    shuffle=False,
    drop_last=False,
    num_workers=8
)

In [18]:
# predictions를 CSV에 저장할 때 형식을 맞춰서 저장
# 테스트 함수 호출
predictions = inference(
    model=model,
    device=device,
    test_loader=test_loader
)

  0%|          | 0/313 [00:00<?, ?it/s]

100%|██████████| 313/313 [05:23<00:00,  1.03s/it]


In [19]:
# 모든 클래스에 대한 예측 결과를 하나의 문자열로 합침
test_info['target'] = predictions
test_info = test_info.reset_index().rename(columns={"index": "ID"})
test_info

,ID,image_path,target
0,0,0.JPEG,328
1,1,1.JPEG,414
2,2,2.JPEG,493
3,3,3.JPEG,17
4,4,4.JPEG,388
...,...,...,...
10009,10009,10009.JPEG,235
10010,10010,10010.JPEG,191
10011,10011,10011.JPEG,466
10012,10012,10012.JPEG,258


In [20]:
# DataFrame 저장
test_info.to_csv("./result/swin2convnext2_384_lr=3e-5padding.csv", index=False)

In [25]:
# torch.save(model.state_dict(), 'model_state_dict_deit_v0=epoch30.pth')